# 단일 PDF 파일 챗봇 샘플 코드

#### Amazon Bedrock의 Knowledge Base 기능 (Chat with your document) 활용
- 이 기능은 현재 Claude 3 Sonnet 과 Claude 3 Haiku 만 지원하고 있습니다.
- 업로드 되는 문서는 10MB 이내여야 합니다.

In [1]:
!pip install boto3 -U

In [2]:
import boto3
import json
import time
import base64

# Bedrock 클라이언트 생성
bedrock_client = boto3.client('bedrock-agent-runtime')


In [3]:
# 문서 파일 경로 

file_name = "Steel Business Briefing_04 Jul 2024.pdf"

#document_uri = f"s3://240605-jesamkim-test-bucket/rag-doc-1/{file_name}"  # S3에 PDF를 업로드 한 경우
file_path = f"./rag_data/{file_name}"                                     # Local PDF 업로드 인 경우

In [4]:
with open(file_path, 'rb') as file:
    pdf_content = file.read()
    pdf_base64 = base64.b64encode(pdf_content).decode('utf-8')

local_sources = [{
    "sourceType": "BYTE_CONTENT",
    "byteContent": {
        "contentType": "application/pdf",
        "data": pdf_base64,
        "identifier": file_name
    }
}]

In [5]:
# 사용자 쿼리

user_query = """
철강 산업 동향에 대해 자세히 알려주세요.
"""

In [8]:
%%time

def retrieveAndGenerate(input_text, sourceType, document_s3_uri=None, data=None, local_sources=None):
    model_arn = "arn:aws:bedrock:us-east-1::foundation-model/anthropic.claude-3-sonnet-20240229-v1:0" # 현재 KB에서는 Claude 3 Sonnet 까지만 지원됩니다.

    if sourceType == "S3":
        return bedrock_client.retrieve_and_generate(
            input={'text': input_text},
            retrieveAndGenerateConfiguration={
                'type': 'EXTERNAL_SOURCES',
                'externalSourcesConfiguration': {
                    'modelArn': model_arn,
                    'sources': [
                        {
                            "sourceType": sourceType,
                            "s3Location": {
                                "uri": document_s3_uri  
                            }
                        }
                    ],
                    'generationConfiguration': {
                        'promptTemplate': {
                            # 시스템 프롬프트 부분
                            'textPromptTemplate': 'You are a chatbot that can explain the key points for domain experts. Answer in Korean.'
                        },
                        'inferenceConfig': {
                            'textInferenceConfig': {
                                'maxTokens': 4096,
                                'temperature': 0.0,
                                'topP': 0.9,
                                'stopSequences': ['\n\nHuman:']
                            }
                        }
                    }
                }
            }
        )
        
    else:
        return bedrock_client.retrieve_and_generate(
            input={'text': input_text},
            retrieveAndGenerateConfiguration={
                'type': 'EXTERNAL_SOURCES',
                'externalSourcesConfiguration': {
                    'modelArn': model_arn,
                    'sources': local_sources
                }
            }
        )

response = retrieveAndGenerate(
    input_text=user_query,
    #sourceType="S3",              # S3에 PDF를 업로드 한 경우
    sourceType="BYTE_CONTENT",     # Local PDF 업로드 인 경우
    #document_s3_uri=document_uri  # S3에 PDF를 업로드 한 경우
    local_sources=local_sources    # Local PDF 업로드 인 경우
    )


# 응답 처리
output = response.get('output', {})
generated_text = output.get('text', '')
print("Generated response: \n", generated_text)



Generated response: 
 최근 철강 산업은 코로나19 팬데믹 이후 경기 회복에 따른 수요 증가와 러시아-우크라이나 전쟁으로 인한 공급 차질 등으로 가격이 큰 폭으로 상승했습니다. 그러나 2022년 하반기부터 경기 둔화 우려와 중국의 제로 코로나 정책 등으로 수요가 감소하면서 철강 가격이 하락세를 보이고 있습니다. 국내 철강업체들은 최근 수출 부진과 내수 경기 둔화로 어려움을 겪고 있습니다. 이에 따라 설비 가동률을 낮추고 비용 절감에 나서는 등 경영 효율화에 주력하고 있습니다. 장기적으로는 친환경 제품 개발과 디지털 전환 등 미래 성장 동력 확보에 힘쓰고 있습니다.
CPU times: user 6.82 ms, sys: 518 μs, total: 7.33 ms
Wall time: 9.6 s


In [9]:
# 인용 정보 출력 (있는 경우)
citations = response.get('citations', [])
if citations:
    print("\nCitations:\n")
    for citation in citations:
        generated_response_part = citation.get('generatedResponsePart', {})
        text_response_part = generated_response_part.get('textResponsePart', {})
        span = text_response_part.get('span', {})
        start = span.get('start', 'N/A')
        end = span.get('end', 'N/A')
        text = text_response_part.get('text', 'N/A')

        #print(f"- Span: {start}-{end}")
        print(f"  Text: {text}")

        retrieved_references = citation.get('retrievedReferences', [])
        for ref in retrieved_references:
            content = ref.get('content', {})
            text = content.get('text', 'N/A')
            location = ref.get('location', {})
            s3_location = location.get('s3Location', {})
            uri = s3_location.get('uri', 'N/A')

            print(f"  Reference:")
            print(f"    URI: {uri}")
            print(f"    Content: {text[:100]}...")  # 처음 100자만 출력
        print()  # 각 citation 사이에 빈 줄 추가


Citations:

  Text: 최근 철강 산업은 코로나19 팬데믹 이후 경기 회복에 따른 수요 증가와 러시아-우크라이나 전쟁으로 인한 공급 차질 등으로 가격이 큰 폭으로 상승했습니다. 그러나 2022년 하반기부터 경기 둔화 우려와 중국의 제로 코로나 정책 등으로 수요가 감소하면서 철강 가격이 하락세를 보이고 있습니다.

  Text: 국내 철강업체들은 최근 수출 부진과 내수 경기 둔화로 어려움을 겪고 있습니다. 이에 따라 설비 가동률을 낮추고 비용 절감에 나서는 등 경영 효율화에 주력하고 있습니다. 장기적으로는 친환경 제품 개발과 디지털 전환 등 미래 성장 동력 확보에 힘쓰고 있습니다.

